Use conda environment `conda_envs/hictk.yml` to run this notebook.

In [1]:
import hictkpy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import itertools
from tqdm import tqdm
from multiprocessing import Pool
import datetime, pytz, time


chroms = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14',
       'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chrX'] + ['trans']


def process_loop(loop):
    a1, a2 = loop
    if loop[0] == loop[1]:
        return np.nan
    else:
        return sort_loop(loop)

def sort_loop(loop):
    a1, a2 = loop

    if chroms.index(a1[0]) > chroms.index(a2[0]):
        return (a2, a1)
    elif chroms.index(a1[0]) == chroms.index(a2[0]):
        if a1[1] > a2[1]:
            return (a2, a1)
        elif a1[1] == a2[1]:
            if a1[2] > a2[2]:
                return (a2, a1)
    return (a1, a2)

# Input data:

- `data/polycomb_dots_hand_coords_update_June25.tsv` - Polycomb dot coordinates identified in this paper
- `/tank/projects/polycomb/data/heffel/cools/*.cool` - snm3C-seq data from *Heffel et al.*
- `/tank/projects/polycomb/results/2024.04.23/metadata.heffel.all_ctx.tsv.gz` - snm3C-seq metadata from *Heffel et al.* 

In [2]:
data_dir = '../../data/'
polycomb_dots = pd.read_csv(data_dir + 'polycomb_dots_hand_coords_update_June25.tsv', sep='\t', index_col=0)

In [3]:
prefix = '/tank/projects/tian2023/'

In [4]:
coolers = os.listdir('/tank/projects/polycomb/data/heffel/cools')
m3C_meta = pd.read_csv('/tank/projects/polycomb/results/2024.04.23/metadata.heffel.all_ctx.tsv.gz', sep='\t', index_col=0)

In [5]:
coolers = m3C_meta.cooler_fpath.to_list()
print(len(coolers))
num_processes = 24
chunk_size = len(coolers) // num_processes
chunks = [coolers[i:i+chunk_size] for i in range(0, len(coolers), chunk_size)]
total=len(coolers) * len(polycomb_dots) // num_processes + 1
print(len(polycomb_dots))

26370
1513


In [6]:
def to_ucsc(reg):
    return f'{reg[0]}:{reg[1]}-{reg[2]}'

def ext(reg, pad=10_000):
    return reg[0], reg[1] - pad, reg[2] + pad

def get_signal(clr, reg1, reg2):
    area = (reg1[2] - reg1[1])//10_000 * (reg2[2] - reg2[1])//10_000 
    signal = clr.fetch(to_ucsc(reg1), to_ucsc(reg2)).sum()
    return signal / area
    
def extract_loops(loops, cool_files):
    data = {}
    
    for f in tqdm(cool_files):
        clr = hictkpy.File(f)
        data[os.path.basename(f)] = {}
        for loop in loops:
            reg1, reg2 = sort_loop(loop)
            signal = get_signal(clr, ext(reg1), ext(reg2))
            data[os.path.basename(f)][loop] = signal
    data = pd.DataFrame(data)
    return data

In [7]:
loops = [eval(l) for l in polycomb_dots.contact_pair.to_list()]

In [8]:
from multiprocessing import Pool, cpu_count
from functools import partial

def extract_loops_single(f):
    data = {}
    clr = hictkpy.File(f)
    data[os.path.basename(f)] = {}
    for loop in loops:
        reg1, reg2 = sort_loop(loop)
        signal = get_signal(clr, ext(reg1), ext(reg2))
        data[os.path.basename(f)][loop] = signal
    data = pd.DataFrame(data)
    return data

def extract_loops_parallel(coolers):
    with Pool(processes=40) as pool:
        results = list(tqdm(pool.imap(extract_loops_single, coolers), 
                            total=len(coolers), desc="Processing Coolers"))
    return results

In [10]:
results = extract_loops_parallel([c for c in coolers])

Processing Coolers: 100%|██████████| 26370/26370 [11:20<00:00, 38.76it/s]


In [11]:
results_t = [result.T for result in results]
results_t_c = pd.concat(results_t)

In [12]:
results_t_c.to_csv(data + 'polycomb_dots_intensities_pad10kb_Heffel.csv')
# gzip-archived in repo